# CRUD
#### (Create,Read,Update,Delete)

### Inhold
1. Start
2. Hent Fra Database
    - Simple Queries
    - Data tranformation
    - Eager loading


----

## 1. Start
for at bruge denne notebook skal du have powershell core installeret.

Winget:

In [ ]:
# Out-file delen kan undværes hvis du copy paster den til din kommandopromt. da den spytter en masse ud.
winget install Microsoft.PowerShell | Out-File -FilePath WingetOutput.txt

Kør koden nedenunder før du kører eksemplerne.

In [ ]:

#!pwsh 
# Powershell kode
Write-Host "Building Project"
dotnet build Efcore.sln | Out-File -FilePath BuildOutput.txt
Write-Host "Project build completed"

#!C#
// Nuget Import 
#r "nuget: Bogus"
#r "nuget: Microsoft.EntityFrameworkCore"
#r "nuget: Microsoft.EntityFrameworkCore.SqlServer"
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite"
#r "DataLayer\bin\Debug\net7.0\DataLayer.dll"

// Usings
using Microsoft.EntityFrameworkCore;
using DataLayer;

// Dbcontext instillinger.
DbContextOptionsBuilder<BlogDbContext> DbOptions = new();
DbOptions.UseSqlite("Data Source=mydb.db;");
//DbOptions.UseSqlServer("<ConnectionString Here>"); // Du kan bruge en normal database hvis du vil.

// Seeding af tabeller
using (var db = new BlogDbContext(DbOptions.Options))
{
    Console.WriteLine("Removing old database..");
    db.Database.EnsureDeleted();
    Console.WriteLine("Creating new database..");
    db.Database.EnsureCreated();
}

// seeding med instillinger
// Alt der bliver oprettet er tilfældige navne og data.
Console.WriteLine("Seeding Database.");
var users = DbOptions.Options.CreateRandomUsers(10); // der vil blive oprettet 10 brugere
var blogs = DbOptions.Options.CreateRandomBlogs(users); // hver bruger ejer en blog
var posts = DbOptions.Options.CreateRandomPosts(40,blogs,users); // 40 post over alle blogs og brugere
Console.WriteLine("Done.");

// Enable Sensitive Data Logging
DbOptions
    .EnableSensitiveDataLogging()
    .LogTo(a => a.Display(),Microsoft.Extensions.Logging.LogLevel.Information);

-----
# 2. Hent fra datbase
## Simple queries
Henter data fra databasen hvor teksten skal indeholde "et".

Der der blibver konverteret til lowercase for at få ikke at søge case sensitiv.

> Note. Det er meningen at Posts og owner er Null.
> Vi kommer til det Senere.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var Blogs = db.Blogs
                    .Where(b => b.BlogName.ToLower().Contains("et"))
                    .ToList();
                    
    Blogs.Display();
}

Her gør vi det samme bare med at vi henter posts med et specifikt Blogid.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var Blogs = db.Posts
                    .Where(b => b.BlogId == 2)
                    .ToList();
                    
    Blogs.Display();
}

## Data Transformering med queries

Man hente data og tranformere til en anden klasse med ``Select()`` og så vil entity framework selv lave de nødvendige sql kommandoer.

> Observer at entity framework selv laver inner joins for at få data

In [ ]:
class BlogPost {
    public string BlogName { get; set; }
    public string UserName { get; set; }
    public string PostTitle { get; set; }
    public string PostDescription { get; set; }
}

using (var db = new BlogDbContext(DbOptions.Options))
{
    var Blogs = db.Posts
                    .Where(p => p.BlogId == 1)
                    .Select(p => new BlogPost {
                        PostTitle = p.title,
                        PostDescription = p.desctription,
                        UserName = p.User.UserName,
                        BlogName = p.Blog.BlogName
                    }).ToList();
                    
    Blogs.Display();
}

## Eager loading

I den første demo var User og Blog Null.

Det er fordi Entityframework kun henter fra den første entitet.


In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var BlogPost = db.Posts
                    .Where(p => p.BlogId == 1)
                    .FirstOrDefault();
                    
    $"Title: {BlogPost?.title}".Display();
    $"Desctription: {BlogPost?.desctription}".Display();
    $"User: {BlogPost.User?.UserName}".Display();
    $"BlogName: {BlogPost.Blog?.BlogName}".Display();
}

Det kan løses ved at bruge Eager loading som vil tilføje entiteneten til query så den kommer med fra databasen ved at bruge ``Include()`` metoden.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var BlogPost = db.Posts
                    .Include(p => p.Blog) // Inkluder Blog
                    .Include(p => p.User) // Inkluder Bruger
                    .Where(p => p.BlogId == 1)
                    .FirstOrDefault();
                    
    $"Title: {BlogPost?.title}".Display();
    $"Desctription: {BlogPost?.desctription}".Display();
    $"User: {BlogPost.User?.UserName}".Display();
    $"BlogName: {BlogPost.Blog?.BlogName}".Display();
}

----
